**Route Optimization**

**List of Authors**
- Arjun Mohankumar
- Kaarthicsudhan Sowripalayam Mani
- Gokul Ram Ellapalayam Rajendran

In [1]:
#import Pkg; Pkg.add("Graphs")
using Graphs

In [2]:
using DelimitedFiles
lines = readdlm("sgb128_dist.txt")
content = lines[1:end,1:end]
content = round.(Int64,content)
#content

128×128 Matrix{Int64}:
    0   966  1513  2964  1149   927  …  1636  2019  1458  1564  2871   348
  966     0  2410  1520  1817   729      558   571   943   198  1917  2541
 1513  2410     0   604   481  2742     1095  2154  1211  2217  2673  1570
 2964  1520   604     0   595  1289      986  2719  1437   769  1038  2343
 1149  1817   481   595     0   494      260  2586  1974  2352  2243   691
  927   729  2742  1289   494     0  …   928  1791  1865  2917   543  1943
 1611   686  1833  1446   550  1279      273  2580   861  2784   249   246
 1510   290   826   466  2641  1197     1294  1917  1807  1989   345   413
  390  1823   214  1139   765  2956     1197   725  1127  2778  1071   405
  466   168  1618   430   934   299     1433  2797   973  2115   281   723
    ⋮                             ⋮  ⋱                       ⋮        
  168  1127   486   861  1994   120     1815  2927  1007  2847  1596   270
  565   432   590   499   324  1429  …   550   710  2154  2154   454  1249
 1700 

In [3]:
#content[91,5]

In [4]:
#import Pkg; Pkg.add("SimpleWeightedGraphs")
using SimpleWeightedGraphs,GraphPlot

In [5]:
g=SimpleWeightedDiGraph(content)
#gplot(g,  edgelabel=weight.(SimpleWeightedGraphs.edges(g)))

{128, 16256} directed simple Int64 graph with Int64 weights

In [6]:
#using LightGraphs

In [7]:
function traverse_graph(
    G::SimpleWeightedDiGraph, 
    J::AbstractArray, 
    start_node::Int, end_node::Int
)
path = Int[start_node]
cost = 0.0
W = SimpleWeightedGraphs.weights(G)

# TODO: step1, initialize v
v = start_node  # CHANGE ME
num = 0
while v != end_node && num < SimpleWeightedGraphs.nv(G)  # prevent infinite loop
    num +=1
    F_v = Graphs.neighbors(G, v)

    # TODO: step 2, compute costs for all n in F_v
    costs = [W[v, n] + J[n] for n in F_v]  # CHANGE ME

    n = F_v[argmin(costs)]

    # TODO: how should we update cost of just the one edge!?
    cost += W[v, n] # CHANGE ME

    push!(path, n)

    # TODO: step 3 -- update v
    v = n  # CHANGE MEa
end
path, cost
end
cost(W, J, n, v) = W[v, n] + J[n]
function compute_J(G::SimpleWeightedDiGraph, dest_node::Int)
    N = SimpleWeightedGraphs.nv(G)
    # step 1. start with zeros
    i = 0
    Ji = zeros(N)

    # allocating memory/creating the array we will 
    # fill in for J_{i+1}
    next_J = zeros(N)

    # get the weight matrix from the graph G
    W = SimpleWeightedGraphs.weights(G)

    done = false
    while !done
        i += 1
        for v in 1:N
            if v == dest_node
                next_J[v] = 0
                continue
            end
            F_v = Graphs.neighbors(G, v)
            costs = [cost(W, Ji, n, v) for n in F_v]
            next_J[v] = minimum(costs)
        end
        done = all(next_J .≈ Ji)
        copy!(Ji, next_J)
    end
    Ji
end
function shortest_path(G::SimpleWeightedDiGraph, start_node::Int, end_node::Int)
    J = compute_J(G, end_node)
    traverse_graph(G, J, start_node, end_node)
end

shortest_path (generic function with 1 method)

In [8]:
#shortest_path(g,1,4)

In [9]:
function remove_hash_lines(input_file::AbstractString)
    # Open the input file in read mode
    open(input_file, "r") do file
        # Read all the lines from the file
        lines = readlines(file)
        
        # Initialize an empty list to store the filtered lines
        filtered_lines = Vector{String}()
        
        # Iterate over each line
        for line in lines
            # Check if the line starts with a hash symbol
            if !startswith(line, '#')
                # If the line doesn't start with a hash, append it to the filtered lines list
                push!(filtered_lines, line)
            end
        end
        
        return filtered_lines
    end
end

# Usage
filtered_lines = remove_hash_lines("sgb128_name.txt")
#filtered_lines

128-element Vector{String}:
 "Youngstown, OH"
 "Yankton, SD"
 "Yakima, WA"
 "Worcester, MA"
 "Wisconsin Dells, WI"
 "Winston-Salem, NC"
 "Winnipeg, MB"
 "Winchester, VA"
 "Wilmington, NC"
 "Wilmington, DE"
 ⋮
 "Richmond, VA"
 "Richmond, IN"
 "Richfield, UT"
 "Rhinelander, WI"
 "Reno, NV"
 "Regina, SA"
 "Red Bluff, CA"
 "Reading, PA"
 "Ravenna, OH"

In [10]:
split_lists = Vector{Vector{String}}()
    
for line in filtered_lines
    # Split the line based on comma (',') and strip whitespace from each element
    split_elements = [strip(element) for element in split(line, ',')]
    
    # Append the split elements to the list of lists
    push!(split_lists, split_elements)
end
#split_lists
map_board=Dict(zip(split_lists,1:length(split_lists)))

Dict{Vector{String}, Int64} with 128 entries:
  ["Utica", "NY"]        => 37
  ["Trenton", "NJ"]      => 46
  ["Waterbury", "CT"]    => 26
  ["Wilmington", "NC"]   => 9
  ["Syracuse", "NY"]     => 56
  ["Salina", "KS"]       => 100
  ["Salem", "OR"]        => 102
  ["Tulsa", "OK"]        => 43
  ["Waterloo", "IA"]     => 25
  ["Richmond", "VA"]     => 120
  ["Reading", "PA"]      => 127
  ["Sioux City", "IA"]   => 72
  ["San Jose", "CA"]     => 90
  ["Santa Fe", "NM"]     => 87
  ["Tallahassee", "FL"]  => 54
  ["Ravenna", "OH"]      => 128
  ["Stroudsburg", "PA"]  => 59
  ["Texarkana", "TX"]    => 51
  ["Rutland", "VT"]      => 112
  ⋮                      => ⋮

In [11]:
#shortest_path(g,map_board[["Utica", "NY"]],map_board[["Trenton", "NJ"]])

In [12]:
 function checkNode(G::SimpleWeightedDiGraph,start,end1,inbetween)
    l = shortest_path(G,start,end1)[1] 
    res = setdiff(Set(inbetween),Set(l))
    collect(res)
 end

checkNode (generic function with 1 method)

In [13]:
#finding number of trucks
#if number of intermediate locations >2
function number_of_trucks(G::SimpleWeightedDiGraph,start,end1,inBetween=[])
    if length(inBetween)==0
        l1=shortest_path(G,start,end1)
        println("Path is ",l1[1])
        println("Cost is ",l1[2])
        println("1 truck is enough")
        return
    end
    #checking any inbetween node is in shortest path between start and end
    c=checkNode(G,start,end1,inBetween)
    if length(c)==0
        l1=shortest_path(G,start,end1)
        println("Path is ",l1[1])
        println("Cost is ",l1[2])
        println("1 truck is enough")
        return
    end
    if length(c)==1 && length(inBetween)==1
        cost1 = [0,0,0]
        cost1[1]=Int(shortest_path(G,start,end1)[2])
        l1 = shortest_path(G,start,inBetween[1])
        c1 = checkNode(G,start,inBetween[1],[end1])
        if length(c1)==0
            println("Path is ",l1[1])
            println("Cost is ",l1[2])
            println("1 truck is enough")
            return
        end
        cost1[2]=Int(l1[2])
        cost1[3]=Int(shortest_path(G,end1,inBetween[1])[2])
        if cost1[2]+cost1[3] < cost1[1]+cost1[2]
            println("Path is ",l1[1]," and ",shortest_path(G,inBetween[1],end1)[1])
            println("Cost is ",cost1[2]+cost1[3])
            println("1 truck is enough")
        else
            println("Path is ",shortest_path(G,start,end1)[1]," and ",l1[1])
            println("Cost is ",cost1[1]+cost1[2])
            println("2 trucks needed")
        end
    # else if length(inBetween)>1
    #     l=[]
    #     for i in inBetween
    #         push!(l,shortest_path(G,start,i))
    #     end
        
    end
end

number_of_trucks (generic function with 2 methods)

In [14]:
#shortest_path(g,1,4)

In [15]:
#the inbetween nodes should be strictly in between start and end and truck has infinite capacity
#number_of_trucks(g,1,4,[94])

In [16]:
#shortest_path(g,1,93)

In [17]:
lines = readdlm("sgb128_dist.txt")
content = lines[1:end,1:end]
content = round.(Int64,content)

g=SimpleWeightedDiGraph(content)

function traverse_graph(
    J::AbstractArray, 
    start_node::Int, end_node::Int
)
path = Int[start_node]
cost = 0.0
W = SimpleWeightedGraphs.weights(g)

# TODO: step1, initialize v
v = start_node  # CHANGE ME
num = 0
while v != end_node && num < SimpleWeightedGraphs.nv(g)  # prevent infinite loop
    num +=1
    F_v = Graphs.neighbors(g, v)

    # TODO: step 2, compute costs for all n in F_v
    costs = [W[v, n] + J[n] for n in F_v]  # CHANGE ME

    n = F_v[argmin(costs)]

    # TODO: how should we update cost of just the one edge!?
    cost += W[v, n] # CHANGE ME

    push!(path, n)

    # TODO: step 3 -- update v
    v = n  # CHANGE MEa
end
path, cost
end
cost(W, J, n, v) = W[v, n] + J[n]
function compute_J(dest_node::Int)
    N = SimpleWeightedGraphs.nv(g)
    # step 1. start with zeros
    i = 0
    Ji = zeros(N)

    # allocating memory/creating the array we will 
    # fill in for J_{i+1}
    next_J = zeros(N)

    # get the weight matrix from the graph G
    W = SimpleWeightedGraphs.weights(g)

    done = false
    while !done
        i += 1
        for v in 1:N
            if v == dest_node
                next_J[v] = 0
                continue
            end
            F_v = Graphs.neighbors(g, v)
            costs = [cost(W, Ji, n, v) for n in F_v]
            next_J[v] = minimum(costs)
        end
        done = all(next_J .≈ Ji)
        copy!(Ji, next_J)
    end
    Ji
end
function shortest_path(start_node::Int, end_node::Int)
    J = compute_J(end_node)
    traverse_graph(J, start_node, end_node)
end


function remove_hash_lines(input_file::AbstractString)
    # Open the input file in read mode
    open(input_file, "r") do file
        # Read all the lines from the file
        lines = readlines(file)
        
        # Initialize an empty list to store the filtered lines
        filtered_lines = Vector{String}()
        
        # Iterate over each line
        for line in lines
            # Check if the line starts with a hash symbol
            if !startswith(line, '#')
                # If the line doesn't start with a hash, append it to the filtered lines list
                push!(filtered_lines, line)
            end
        end
        
        return filtered_lines
    end
end

# Usage
filtered_lines = remove_hash_lines("sgb128_name.txt")
#filtered_lines


split_lists = Vector{Vector{String}}()
    
for line in filtered_lines
    # Split the line based on comma (',') and strip whitespace from each element
    split_elements = [strip(element) for element in split(line, ',')]
    
    # Append the split elements to the list of lists
    push!(split_lists, split_elements)
end
#split_lists
map_board=Dict(zip(split_lists,1:length(split_lists)))
map_board_rev=Dict()
for i in eachindex(map_board)
    println(i,"=>",map_board[i])
    map_board_rev[map_board[i]]=i
end
function checkNode(start,end1,inbetween)
    l = shortest_path(start,end1)[1] 
    res = setdiff(Set(inbetween),Set(l))
    collect(res)
 end

#finding number of trucks
#if number of intermediate locations >2
function number_of_trucks1(start,end1,inBetween=[])
    if length(inBetween)==0
        l1=shortest_path(start,end1)
        println("Path is ",l1[1])
        println("Cost is ",l1[2])
        println("1 truck is enough")
        return l1,([""],1)
    end
    #checking any inbetween node is in shortest path between start and end
    c=checkNode(start,end1,inBetween)
    if length(c)==0
        l1=shortest_path(start,end1)
        println("Path is ",l1[1])
        println("Cost is ",l1[2])
        println("1 truck is enough")
        return l1,([""],1)
    end
    if length(c)==1 && length(inBetween)==1
        cost1 = [0,0,0]
        cost1[1]=Int(shortest_path(start,end1)[2])
        l1 = shortest_path(start,inBetween[1])
        c1 = checkNode(start,inBetween[1],[end1])
        if length(c1)==0
            println("Path is ",l1[1])
            println("Cost is ",l1[2])
            println("1 truck is enough")
            return l1,([""],1)
        end
        cost1[2]=Int(l1[2])
        cost1[3]=Int(shortest_path(end1,inBetween[1])[2])
        if cost1[2]+cost1[3] < cost1[1]+cost1[2]
            println("Path is ",l1[1]," and ",shortest_path(inBetween[1],end1)[1])
            println("Cost is ",cost1[2]+cost1[3])
            println("1 truck is enough")
            return l1,shortest_path(inBetween[1],end1),([""],1)
        else
            println("Path is ",shortest_path(start,end1)[1]," and ",l1[1])
            println("Cost is ",cost1[1]+cost1[2])
            println("2 trucks needed")
            return shortest_path(start,end1),l1
        end
    # else if length(inBetween)>1
    #     l=[]
    #     for i in inBetween
    #         push!(l,shortest_path(G,start,i))
    #     end
        
    end
end


["Utica", "NY"]=>37


["Trenton", "NJ"]=>46
["Waterbury", "CT"]=>26
["Wilmington", "NC"]=>9
["Syracuse", "NY"]=>56
["Salina", "KS"]=>100
["Salem", "OR"]=>102
["Tulsa", "OK"]=>43
["Waterloo", "IA"]=>25
["Richmond", "VA"]=>120
["Reading", "PA"]=>127
["Sioux City", "IA"]=>72
["San Jose", "CA"]=>90
["Santa Fe", "NM"]=>87
["Tallahassee", "FL"]=>54
["Ravenna", "OH"]=>128
["Strouds

burg", "PA"]=>59
["Texarkana", "TX"]=>51
["Rutland", "VT"]=>112
["Rock Springs", "WY"]=>115
["Saginaw", "MI"]=>110
["Vancouver", "BC"]=>34
["Saint Louis", "MO"]=>104
["Wheeling", "WV"]=>16
["Springfield", "OH"]=>65
["Red Bluff", "CA"]=>126
["Traverse City", "MI"]=>47
["Sumter", "SC"]=>58
["Salisbury", "MD"]=>98
["San Angelo", "TX"]=>96
["Waycross", "GA"]=>20
["Sheridan", "WY"]=>75
["Waco", "TX"]=>30
["Tucson", "AZ"]=>44
["Sherman", "TX"]=>74
["Wichita", "KS"]=>15
["Washington", "DC"]=>27
["Schenectady", "NY"]=>82
["Watertown", "SD"]=>23
["Tuscaloosa", "AL"]=>41
["Regina", "SA"]=>125
["Williamsport", "PA"]=>12
["Richfield", "UT"]=>122
["Toronto", "ON"]=>48
["Springfield", "MA"]=>67
["Seminole", "OK"]=>76
["Rocky Mount", "NC"]=>114
["Reno", "NV"]=>124
["Watertown", "NY"]=>24
["Shreveport", "LA"]=>73
["Roswell", "NM"]=>113
["San Diego", "CA"]=>93
["Trinidad", "CO"]=>45
["Warren", "PA"]=>28
["Twin Falls", "ID"]=>40
["Savannah", "GA"]=>83
["Worcester", "MA"]=>4
["Seattle", "WA"]=>79
["Tyler

number_of_trucks1 (generic function with 2 methods)

In [18]:
x1=number_of_trucks1(1,4,[94])

Path is [1, 94] and [94, 5, 93, 4]
Cost is 487
1 truck is enough


(([1, 94], 130.0), ([94, 5, 93, 4], 357.0), ([""], 1))

In [19]:
trucks = number_of_trucks1(1,4,[94])
x11=[]
for i in 1:length(trucks)-1
    for j in trucks[i][1]
        push!(x11,j)
    end
end
r11=[]
push!(r11,0)
for i in 2:length(x11)
    if x11[i-1]==x11[i]
        push!(r11,0)
        continue
    end
    push!(r11,content[x11[i-1],x11[i]])
end
r11
sum(r11)

Path is [1, 94] and [94, 5, 93, 4]
Cost is 487
1 truck is enough

487

In [20]:
content[93,5]

213

In [21]:
map_board_rev=Dict()

Dict{Any, Any}()

In [22]:
map_board_rev=Dict()
for i in eachindex(map_board)
    println(i,"=>",map_board[i])
    map_board_rev[map_board[i]]=i
end
map_board_rev

["Utica", "NY"]=>37
["Trenton", "NJ"]=>46
["Waterbury", "CT"]=>26
["Wilmington", "NC"]=>9
["Syracuse", "NY"]=>56
["Salina", "KS"]=>100
["Salem", "OR"]=>102
["Tulsa", "OK"]=>43
["Waterloo", "IA"]=>25
["Richmond", "VA"]=>120
["Reading", "PA"]=>127
["Sioux City", "IA"]=>72
["San Jose", "CA"]=>90
["Santa Fe", "NM"]=>87
["Tallahassee", "FL"]=>54
["Ravenna", "OH"]=>128
["Stroudsburg", "PA"]=>59
["Texarkana", "TX"]=>51
["Rutland", "VT"]=>112
["Rock Springs", "WY"]=>115
["Saginaw", "MI"]=>110
["Vancouver", "BC"]=>34
["Saint Louis", "MO"]=>104
["Wheeling", "WV"]=>16
["Springfield", "OH"]=>65
["Red Bluff", "CA"]=>126
["Traverse City", "MI"]=>47
["Sumter", "SC"]=>58
["Salisbury", "MD"]=>98
["San Angelo", "TX"]=>96
["Waycross", "GA"]=>20
["Sheridan", "WY"]=>75
["Waco", "TX"]=>30
["Tucson", "AZ"]=>44
["Sherman", "TX"]=>74
["Wichita", "KS"]=>15
["Washington", "DC"]=>27
["Schenectady", "NY"]=>82
["Watertown", "SD"]=>23
["Tuscaloosa", "AL"]=>41
["Regina", "SA"]=>125
["Williamsport", "PA"]=>12
["Richfi

ton", "VA"]=>64
["Sault Sainte Marie", "MI"]=>84
["Winnipeg", "MB"]=>7
["Stockton", "CA"]=>60
["Weed", "CA"]=>19
["Salt Lake City", "UT"]=>97
["Toledo", "OH"]=>50
["Walla Walla", "WA"]=>29
["Sandusky", "OH"]=>92
["Sedalia", "MO"]=>78
["Saint Joseph", "MI"]=>106
["Vicksburg", "MS"]=>33
["Wenatchee", "WA"]=>18
["Waukegan", "IL"]=>22
["Youngstown", "OH"]=>1
["Tacoma", "WA"]=>55
["Santa Ana", "CA"]=>89
["Salinas", "CA"]=>99
["Saint Joseph", "MO"]=>105
["Williston", "ND"]=>11
["San Francisco", "CA"]=>91
["Yankton", "SD"]=>2
["San Bernardino", "CA"]=>94
["Rhinelander", "WI"]=>123
["Saint Augustine", "FL"]=>109
["Rockford", "IL"]=>116
["Victoria", "TX"]=>32
["Vincennes", "IN"]=>31
["West Palm Beach", "FL"]=>17
["Sioux Falls", "SD"]=>71
["Roanoke", "VA"]=>119
["Rochester", "MN"]=>118
["Swainsboro", "GA"]=>57
["Terre Haute", "IN"]=>52
["Tampa", "FL"]=>53
["Selma", "AL"]=>77


Dict{Any, Any} with 128 entries:
  5   => ["Wisconsin Dells", "WI"]
  56  => ["Syracuse", "NY"]
  114 => ["Rocky Mount", "NC"]
  35  => ["Valley City", "ND"]
  110 => ["Saginaw", "MI"]
  55  => ["Tacoma", "WA"]
  123 => ["Rhinelander", "WI"]
  60  => ["Stockton", "CA"]
  30  => ["Waco", "TX"]
  32  => ["Victoria", "TX"]
  6   => ["Winston-Salem", "NC"]
  67  => ["Springfield", "MA"]
  45  => ["Trinidad", "CO"]
  117 => ["Rochester", "NY"]
  73  => ["Shreveport", "LA"]
  115 => ["Rock Springs", "WY"]
  112 => ["Rutland", "VT"]
  64  => ["Staunton", "VA"]
  90  => ["San Jose", "CA"]
  ⋮   => ⋮

In [23]:
l=copy(keys(map_board))
l=collect(l)
insert!(l, 1, [""])

129-element Vector{Vector{String}}:
 [""]
 ["Utica", "NY"]
 ["Trenton", "NJ"]
 ["Waterbury", "CT"]
 ["Wilmington", "NC"]
 ["Syracuse", "NY"]
 ["Salina", "KS"]
 ["Salem", "OR"]
 ["Tulsa", "OK"]
 ["Waterloo", "IA"]
 ⋮
 ["Vincennes", "IN"]
 ["West Palm Beach", "FL"]
 ["Sioux Falls", "SD"]
 ["Roanoke", "VA"]
 ["Rochester", "MN"]
 ["Swainsboro", "GA"]
 ["Terre Haute", "IN"]
 ["Tampa", "FL"]
 ["Selma", "AL"]